In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error


In [2]:
df = pd.read_csv("car_details.csv")
df.sample(5)

,name,company,model,edition,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
1988,Maruti Wagon R LXI Minor,Maruti,Wagon,R LXI Minor,2009,Fourth & Above,Petrol,Individual,Manual,120000,44.40,1061.0,67.00,84.0,5.0,125000
1763,Datsun RediGO T,Datsun,RediGO,T,2017,First,Petrol,Individual,Manual,25000,53.34,799.0,53.64,72.0,5.0,300000
1130,Maruti Ciaz VDi Plus SHVS,Maruti,Ciaz,VDi Plus SHVS,2015,First,Diesel,Individual,Manual,84000,66.00,1248.0,88.50,200.0,5.0,650000
5956,Hyundai Verna 1.6 CRDI,Hyundai,Verna,1.6 CRDI,2011,First,Diesel,Individual,Manual,108000,52.44,1582.0,126.30,259.9,5.0,450000
1800,Maruti Vitara Brezza ZDi,Maruti,Vitara,Brezza ZDi,2018,First,Diesel,Dealer,Manual,58511,57.10,1248.0,88.50,200.0,5.0,950000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [4]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [5]:
df.shape

(6926, 16)

In [12]:
for col in df.select_dtypes(include="O").columns:
    print(f"Column: {col}")
    print(f"Cardinality: {df[col].nunique()}")
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))
    print()

Column: name
Cardinality: 2058
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                          0.017037
Maruti Alto 800 LXI                             0.010973
Maruti Alto LXi                                 0.009962
Maruti Swift VDI                                0.008663
Maruti Swift VDI BSIV                           0.008085
                                                  ...   
Mahindra KUV 100 G80 K4 Plus                    0.000144
Hyundai Getz 1.1 GVS                            0.000144
Maruti 800 EX BSII                              0.000144
Datsun GO T Option                              0.000144
Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV    0.000144
Name: proportion, Length: 2058, dtype: float64

Column: company
Cardinality: 32
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' '

In [13]:
df = df.drop(columns=["name","model","edition"])

In [ ]:
df.duplicated().sum()

19

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.duplicated().sum()

0

In [18]:
X = df.drop(columns="selling_price")
y = df.selling_price.copy()

print(X.shape, y.shape)

(6907, 12) (6907,)


In [20]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(5525, 12) (5525,)
(1382, 12) (1382,)


In [23]:
num_cols = X_train.select_dtypes(include="number").columns.tolist()
cat_cols = [col for col in X_train.columns if col not in num_cols]
print(num_cols, cat_cols)

['year', 'km_driven', 'mileage_mpg', 'engine_cc', 'max_power_bhp', 'torque_nm', 'seats'] ['company', 'owner', 'fuel', 'seller_type', 'transmission']


In [27]:
num_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

cat_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols )
    ]
)

regressor = RandomForestRegressor(
    n_estimators=10,
    max_depth= 5,
    random_state= 42
)

rf_model = Pipeline(steps=[
    ("pre", preprocessor),
    ("reg", regressor)
])

rf_model.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'km_driven',
                                                   'mileage_mpg', 'engine_cc',
                                                   'max_power_bhp', 'torque_nm',
                                                   'seats']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['company', 'owner', 'fuel',
                                                   'seller_type',
                                                   'transmission'])])),
                ('reg',
                 RandomForestRegressor(max_depth=5, n_estimators=10,
                                       random_state=42))])